In [1]:
from insightface.app import FaceAnalysis
import cv2
import tensorflow as tf

In [2]:
app = FaceAnalysis(allowed_modules=["detection"])
app.prepare(ctx_id=0, det_size=(640, 640))

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /home/idk/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /home/idk/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/idk/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /home/idk/.insightface/models/buffalo_l/genderage.onnx genderage
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /home/idk/.insightface/models/buffalo_l/w600k_r50.onnx recognition
set det-size: (640, 640)


In [3]:
det = app.det_model

input_std = det.input_std
input_mean = det.input_mean
session = det.session
output_names = det.output_names
input_name = det.input_name
fmc = det.fmc
feat_stride_fpn = det._feat_stride_fpn
use_kps = det.use_kps
num_anchors = det._num_anchors
threshold = 0.5
img = cv2.imread("example.jpg")
input_size = 640
image = cv2.resize(img, (640, 640))
blob = cv2.dnn.blobFromImage(
    image,
    1.0 / input_std,
    (input_size, input_size),
    (input_mean, input_mean, input_mean),
    swapRB=True,
)
net_outs = session.run(output_names, {input_name: blob})


In [16]:
def distance2bbox(points, distance):
    x1 = points[:, :, 0] - distance[:, :, 0]
    y1 = points[:, :, 1] - distance[:, :, 1]
    x2 = points[:, :, 0] + distance[:, :, 2]
    y2 = points[:, :, 1] + distance[:, :, 3]
    return tf.stack([x1, y1, x2, y2], axis=-1)


repeats = 2
outputs = list(
    map(lambda x: tf.repeat(tf.expand_dims(x, 0), repeats, axis=0), net_outs)
)
boxes_final = []
scores_final = []
batch_size = repeats

for idx, stride in enumerate(feat_stride_fpn):
    scores = tf.squeeze(outputs[idx], -1)

    bbox_preds = outputs[idx + fmc]
    bbox_preds = bbox_preds * stride
    height = input_size // stride
    width = input_size // stride
    anchor_centers = tf.stack(
        tf.meshgrid(tf.range(height), tf.range(width), indexing="ij")[::-1], axis=-1
    )
    anchor_centers = tf.cast(anchor_centers, tf.float32)
    anchor_centers = tf.reshape(anchor_centers * stride, (-1, 2))

    if num_anchors > 1:
        anchor_centers = tf.reshape(
            tf.stack([anchor_centers] * num_anchors, axis=1), (-1, 2)
        )

    anchor_centers = tf.expand_dims(anchor_centers, 0)
    pos_inds = scores >= threshold
    bboxes = distance2bbox(anchor_centers, bbox_preds)

    pos_scores = tf.ragged.boolean_mask(scores, pos_inds)
    pos_bboxes = tf.ragged.boolean_mask(bboxes, pos_inds)

    boxes_final.append(pos_bboxes)
    scores_final.append(pos_scores)


scores_cat = tf.concat(scores_final, 1)
bboxes_cat = tf.concat(boxes_final, 1)
masks = []

scores_good = []
bboxes_good = []
for i in range(batch_size):
    mask = tf.image.non_max_suppression(
        bboxes_cat[i], scores_cat[i], max_output_size=100
    )
    bboxes_good.append(tf.gather(bboxes_cat[i], indices=mask))
    scores_good.append(tf.gather(scores_cat[i], indices=mask))


res_scores = tf.ragged.stack(scores_good)
res_bboxes = tf.ragged.stack(bboxes_good)


In [46]:
mask

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([3], dtype=int32)>